<a href="https://colab.research.google.com/github/fanny/information-retrieval/blob/master/lab3/tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import re

import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('https://raw.githubusercontent.com/fanny/information-retrieval/master/lab2/data/results.csv')
data.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,_id,author,date,section,sub_title,text,title,url
0,7,NaN,05/04/2019 |,agronegocio,NaN,NaN,\t\t\t\t,https://www.gazetadopovo.com.br/agronegocio/ex...
1,2,NaN,NaN,wiseup-news,NaN,NaN,NaN,https://www.gazetadopovo.com.br/wiseup-news/bo...
2,3,NaN,NaN,futebol,NaN,NaN,NaN,https://especiais.gazetadopovo.com.br/futebol/...
3,9,NaN,05/04/2019 |,agronegocio,NaN,NaN,Suspensão de frigoríficos não impediu alta de ...,https://www.gazetadopovo.com.br/agronegocio/ag...
4,1,The Daily Signal,NaN,ideias,NaN,Pessoas que buscam uma identidade transsexual ...,Populares,https://www.gazetadopovo.com.br/ideias/nao-se-...
5,5,Gazeta do Povo,NaN,curitiba,NaN,"Depois de uma semana de calor, Curitiba terá u...",Populares,https://www.gazetadopovo.com.br/curitiba/previ...
6,40,"Flávia Alves, especial para a Gazeta do Povo",NaN,viver-bem,NaN,As promessas são muitas: combater o envelhecim...,NaN,https://www.gazetadopovo.com.br/viver-bem/saud...
7,6,Agência Estado,NaN,republica,NaN,"O líder do MDB na Câmara, Baleia Rossi (SP), p...",Populares,https://www.gazetadopovo.com.br/republica/pec-...
8,20,Daniel Malucelli,[05/04/2019],esportes,mercado,O meia-atacante Maicosuel está com a situação ...,Paraná não descarta dispensa de Maicosuel ante...,https://www.gazetadopovo.com.br/esportes/paran...
9,10,Renyere Trovão,NaN,automoveis,NaN,Concretizar a compra de um carro novo nem semp...,Populares,https://www.gazetadopovo.com.br/automoveis/car...


In [0]:
clean_text = data['text'].dropna()
def filter_by_criteria(text, criteria):
  filtered_words = re.findall(criteria, text)
  filtered_words = ' '.join(filtered_words)
 
  return filtered_words

In [0]:
def get_words_filtered(criteria):
  filtered_text = clean_text.apply(lambda text: filter_by_criteria(text, criteria))
  all_filtered_text = filtered_text.str.cat(sep=' ')
  all_words_filtered = all_filtered_text.split(' ')

  all_words_filtered = [text for text in all_words_filtered if len(text) > 0]

  return set(all_words_filtered[:200])

## Tokenização

> Tokenização é o processo de formar palavras de uma sequência de caracteres em um documento    

**Obs: Texto extraido do livro**

E para formar essas palavras podemos usar alguns critérios, como:
- Quantidade de caracteres; Palavras hifenizadas; Caracteres especiais;
- Caracteres em Caixa Alta ou Baixa; Apostrófos; Números; Abreviações;

Cada critério desse, pode guardar uma importância, se tratando dessa base de dados de notícias, alguns poderão ser mais relevantes que outros e veremos o motivo.

Para conseguir concluir quais critérios usar no processo de tokenização, começaremos analisando os dados, garantindo assim que não ignoremos palavras que são importantes.

Se você quiser pular essa parte de análise, e ir para os critérios de tokenização que irei considerar, [clique aqui](https://colab.research.google.com/drive/1EP_hx0xDOgID34aTHwCNL-AFgBWvrGcH?authuser=1#scrollTo=hz-6d5cp6zEJ), mas tenha em mente que só explico os fatores que me levaram a tomar essa decisão aqui embaixo.

### Quantidade de caracteres

A primeira suposição é que palavras com quantidade menores que 3 geralmente não são importantes, porque representam artigos, preposições etc, que não agregam tanto valor na busca.

#### Resultados

In [0]:
get_words_filtered(r'\b\w{0,3}\b')


A título de informação, só mostramos o set das 200 primeiras, mas olhando os dados podemos observar coisas interessantes como:

- Palavras capitalizadas indicam Siglas - APA, PIB, IPI, IVA, PEC, RJ
- Números só com um dígito, não são relevantes
- Ocorrem palavras com tamanho menor ou igual a dois que são importantes, indicam siglas, como 'RJ', 'PEC' e outras que o seu próprio significado tem muito valor, como 'gay' e 'pai'

Sendo assim, removeremos as palavras de tamanho 1 e as stopwords do brasil.


### Palavras hífenizadas

Provavelmente palavras com hífen serão muito importantes, porque elas representam dias da semana, nomes geográficos(Grã-Bretanha), certas espécies de animais e vegetais(Bem-te-vi, Couve-Flor) alguns adjetivos (mal-humorada) dentre outras.

In [0]:
get_words_filtered(r'[a-zA-Z]+\-[a-zA-Z]+')


De fato, foram encontrados dias da semana, partidos, cargos, e muitas palavras importantes com hífen demonstrando que elas não devem ser removidas.

### Caracteres especiais

Também podem ser importantes, basta pensarmos em '%' que pode indicar percentuais, '$' que pode indicar dinheiro, 'º' que pode inficar uma posição, enfim, veremos a seguir, se esses símbolos são importantes no texto.

Olhando símbolos nos textos, além da nossa suposição, foram encontrados certos delimitadores, que não acrescentam informações no texto como:
- '\r\n'
- '\xaO'  
Sendo assim, removeremos eles.

### Apóstrofos

Podem indicar marcas (Burguer’s King, McDonald’s) extrangeiras, ou uma “composição” de palavras como: pingo d’ água

#### Resultados

In [0]:
get_words_filtered(r"\w+\'\w*")

As palavras encontradas com apóstrofo,  provavlmente são erros de digitação/formatação, fazendo mais sentido da replace pra um caractere vazio, e não considerar o hífen na tokenização.

### Números

Números são extremamente importantes, podem indicar horas, salários, anos, posições, temperaturas, etc.

#### Resultados


In [0]:
get_words_filtered(r'\d+\w+')

Confirmando a suposição, não é interessante removê-los.

### Abreviações

Podem agregar muito valor pois servem para indicar cidades, siglas e etc..

### Resultados

In [0]:
get_words_filtered(r'(\w+\.\w+)*')

Assim, como no caso anterior, não é bom que eliminemos, pois temos números decimais, domínios de sites e etc.

## Conlusões dos critérios de tokenização

Da análise anterior, fica claro, que removeremos apóstrofos, palavras de tamanho igual a 1, stopwords e alguns símbolos de escape.

## Criando os tokens



In [138]:
all_text = clean_text.str.cat(sep=' ')
hyphenated_and_abreviated_words = '\w{2,}[\-*|\.*]\w+'
decimal_numbers_with_symbols = '\w?\d{2,}[\.|\,]*\d+\W*'
words ='\w{2,}'
numbers_with_symbols = '\w?\W*\d{2,}\W*'
word_list = regexp_tokenize(all_text, pattern='%s|%s|%s|%s' %(hyphenated_and_abreviated_words, decimal_numbers_with_symbols, words, numbers_with_symbols))
word_list = pd.DataFrame(word_list, columns=['word'])
word_list.head(30)

,word
0,Pessoas
1,que
2,buscam
3,uma
4,identidade
5,transsexual
6,não
7,nasceram
8,assim
9,crianças


### Entendendo regex



```regex
\w{2,}[\-*|\.*]\w+|\d{2,}[\.*|\,*]\d+|\w{2,}|\d{2,}'

```
Quebramos em tokens pelos seguintes critérios:
- Palavras com tamanho maior que 2: `\w{2,}`
- Números com quant. de algarismos maior que 2 com símbolos: `\w?\W*\d{2,}\W*`
- Palavras hifenizadas, ou com ponto:`\w{2,}[\-*|\.*]\w+`
- Números decimais com símbolos('$', '%'): `\w?\d{2,}[\.|\,]*\d+\W*`

